

# Intoduction

In order to build my deep learning model that can recognize 6 categories of natural images I used the pretrained GoogleNet (Inception) model.

Since the my images training set were store in a folder on the drive I first had to create my own customized dataset to that will be used to train my model and an excel sheet (or CSV file) that will hold all the images names and its associated class (label).

Afterwards, the model was trained using the training set provided, and the best model (least loss) was stored. In order to facilitate testing the model generated I used two methods to predict the class of any given image from the testing images.

Finally, all the images in the tesing folder were stored in an excel sheet with the prediction generated by the model.


##Imports

Importing all the libraries needed

In [ ]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import seaborn as sns
import numpy as np
from torch.utils.data import random_split
from pandas.tseries.offsets import YearEnd
import os
import pandas as pd
from skimage import io, transform
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import utils
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16
import cv2
import copy
import csv




In [ ]:
drive.mount('/content/gdrive')

dic = {'buildings' : 0,
        'forest': 1,
        'glacier': 2,
        'mountain': 3,
        'sea': 4,
        'street' : 5
}


wtr = csv.writer(open ('images.csv', 'w'), delimiter=',', lineterminator='\n')
wtr.writerow(["Image Name", "class"])
for key,value in dic.items():
  arr = os.listdir(f'gdrive/MyDrive/Scenes training set/{key}')  
  for x in arr:
    wtr.writerow ([x,value])

wtr = csv.writer(open ('images.csv', 'w'), delimiter=',', lineterminator='\n')
wtr.writerow(["Image Name", "class"])
for key,value in dic.items():
  arr = os.listdir(f'gdrive/MyDrive/Scenes training set/{key}')  
  for x in arr:
    wtr.writerow ([x,value])

  



Mounted at /content/gdrive


##Building my dataset

In [ ]:
d = {0:'buildings' ,
       1: 'forest',
        2:'glacier',
        3:'mountain',
        4:'sea',
        5:'street'
}
              

class ImagesDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform,dtype):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform =  transform
        self.dtype = dtype
        

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if self.dtype=='train':
          img_name = os.path.join(self.root_dir,d[self.landmarks_frame.iloc[idx, 1]],self.landmarks_frame.iloc[idx, 0])
        else:
          img_name = os.path.join(self.root_dir,self.landmarks_frame.iloc[idx, 0])
        image = Image.open(img_name)
        label = torch.tensor(int(self.landmarks_frame.iloc[idx, 1]))

        if self.transform:
          image = self.transform(image)
        
        return image,label



## Training my model

Specified the size of each batch of training examples utilized in one iteration (batch size), the learning rate and number of possible classes it was time to train the model.

In [ ]:
BATCH_SIZE=64
num_epochs=3
lr=1e-4
class_size=6
in_channel=3

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform_train = transforms.Compose([transforms.Resize((224,224)), transforms.RandomHorizontalFlip(p=0.7), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
transform_test = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


train = ImagesDataset(csv_file='images.csv', root_dir='gdrive/MyDrive/Scenes training set' , transform=transform_train,dtype='train')

#  train, val and test datasets to the dataloader
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

model =torchvision.models.resnet50(pretrained=True)
model.fc = nn.Linear(2048, class_size)

#model1 =torchvision.models.mobilenet_v2(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= lr) 
loss=0
min_loss=1000000

for epoch in range(num_epochs): #I decided to train the model for 50 epochs
    loss_var = 0

    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device=device)
        labels = labels.to(device=device)
        ## Forward Pass
        scores = model(images)
        loss = criterion(scores,labels)

        ## Backward Pass
        optimizer.zero_grad()
        loss.backward()


        optimizer.step()
        loss_var = loss.item()

        loss=loss_var/len(train_loader)

        if idx%64==0:
            print(f'Epoch [{epoch+1}/{num_epochs}] || Step [{idx+1}/{len(train_loader)}] || Loss:{loss}')
    print(f"Loss at epoch {epoch+1} || {loss_var/len(train_loader)}")

NameError: ignored

In [ ]:
torch.save(model.state_dict(), "model.pt") #SAVES THE TRAINED MODEL
model.load_state_dict(torch.load("model.pt")) #loads the trained model
model.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

## Prediction functions

The following functions are used to predict the output class; given an input and a trained model.

In [ ]:
def predict(tensor, model):
    yhat = model(tensor.unsqueeze(0))
    yhat = yhat.clone().detach()
    return yhat


def predict_class(x): 
  tr1 = transforms.ToTensor()
  tr2 = transforms.ToPILImage()

  image = io.imread(f'gdrive/MyDrive/Scenes testing test/{x}')
  image = transform_train(tr2( tr1(image)))
  yhat = predict(image , model)
  label = yhat.max(1)
  indx = int(label[1][0])
  return indx


In [ ]:
wtr2 = csv.writer(open ('testimages.csv', 'w'), delimiter=',', lineterminator='\n')
wtr2.writerow(["Image Name", "class"])
arr = os.listdir(f'gdrive/MyDrive/Scenes testing test')  

for x in arr:
  y=predict_class(x)
  wtr2.writerow ([x,y])

wtr2 = csv.writer(open ('testimages.csv', 'w'), delimiter=',', lineterminator='\n')
wtr2.writerow(["Image Name", "class"])


for x in arr:
  y=predict_class(x)
  wtr2.writerow ([x,y])
  